# Init

### Required

In [ ]:
!wget https://raw.githubusercontent.com/evhenyshaliov/ecim/main/requirements.txt
!wget https://raw.githubusercontent.com/evhenyshaliov/ecim/main/image.jpeg
!pip install -r requirements.txt

In [1]:
import pprint
import requests

import os
import pprint
import urllib3
import json

from pprint import pprint

from osdu_api.auth.refresh_token import BaseTokenRefresher

from osdu_api.clients.storage.record_client import RecordClient
from osdu_api.clients.search.search_client import SearchClient
from osdu_api.clients.schema.schema_client import SchemaClient
from osdu_api.clients.dataset.dataset_dms_client import DatasetDmsClient
from osdu_api.clients.dataset.dataset_registry_client import DatasetRegistryClient

from osdu_api.model.storage.legal import Legal
from osdu_api.model.storage.record import Record
from osdu_api.model.storage.record_ancestry import RecordAncestry
from osdu_api.model.storage.acl import Acl
from osdu_api.model.search.query_request import QueryRequest
from osdu_api.model.storage.query_records_request import QueryRecordsRequest
from osdu_api.model.dataset.create_dataset_registries_request import CreateDatasetRegistriesRequest

urllib3.disable_warnings()


#### Option 1: Credentials

In [ ]:
from google.colab import auth
auth.authenticate_user()

#### Option 2: Service Account

In [ ]:
os.environ["SA_FILE_PATH"] = "osdu-ecim-demo.json"

#### Loading the config

In [ ]:
BASE_URL = "https://ecim.gcp.gnrg-osdu.projects.epam.com"
CLOUD_PROVIDER = "gc"
os.environ["CLOUD_PROVIDER"] = CLOUD_PROVIDER

config_url = BASE_URL + "/api/config/v1/postman-environment"
config_response = requests.get(config_url).json()
config = {}
for entity in config_response['values']:
  config[entity['key']] = entity['value']

# pprint(config_response)
# print(config['SEISMICDMS_HOST'])

In [ ]:
token_refresher = BaseTokenRefresher()
token_refresher.refresh_token()

#### OSDU clients and default values

In [ ]:

search_client = SearchClient(
    search_url=f"https://{config['SEARCH_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

schema_client = SchemaClient(
    schema_url=f"https://{config['SCHEMA_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

dataset_client = DatasetDmsClient(
    dataset_url=f"https://{config['DATASET_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)
dataset_registry_client = DatasetRegistryClient(
    dataset_url=f"https://{config['DATASET_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

storage_client = record_client = RecordClient(
    storage_url=f"https://{config['STORAGE_HOST']}",
    provider=CLOUD_PROVIDER,
    token_refresher=token_refresher,
    data_partition_id=config['data-partition-id']
)

query_request = QueryRequest(
    kind="osdu:wks:dataset--File.Generic:1.0.0",
    query="",
    limit=1,
    returned_fields=["*"]
)
response = search_client.query_records(query_request).json()
default_acl = response['results'][0]['acl']
default_legal_tags = ["osdu-default-data-tag"]

# pprint(response)
pprint(default_acl)
pprint(default_legal_tags)

urllib3.disable_warnings()

# Slot 1 - Using Core APIs

#### All to register and make successful connection to workbook and both OSDU instances

In [ ]:
search_response = requests.get(f"https://{config['SEARCH_HOST']}/info")

pprint(search_response.json())

#### All able to execute and understand the result on some of the core APIs

In [ ]:
# List all records
query_request = QueryRequest(
    kind= "*:*:*:*",
    query="",
    limit=2,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())


In [ ]:
# List a specific subset of records based on schema type/kind
query_request = QueryRequest(
    kind="osdu:wks:dataset--File.Generic:1.0.0",
    query="",
    limit=2,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Find the description of specific schema/kind to show that schemas are a part of the platform
response = schema_client.get_schema_by_id('').json()
pprint(response)

In [ ]:
# Extract some data from a specific data type
# ???

#### Show that the platform is generic and/extendable and can store any data type in addition to defined "well known" schemas

In [ ]:
response = dataset_client.storage_instructions(kind_sub_type='dataset--File.Generic').json()

pprint(response)

In [ ]:
# All to upload an image

def upload_file_and_metadata():
  storage_instruction = dataset_client.storage_instructions(kind_sub_type="dataset--File.Generic")
  storage_location = storage_instruction.json()['storageLocation']
  #### Uploading
  with open('image.jpeg', 'rb') as file_content:
    new_file = dataset_client.put_file(url=storage_location['signedUrl'], data=file_content, no_auth=True)

  record_list = [
                  Record( kind='osdu:wks:dataset--File.Generic:1.0.0',
                          acl=Acl(owners=default_acl['owners'], viewers=default_acl['viewers']),
                          legal=Legal(legaltags=default_legal_tags, other_relevant_data_countries=["US"], status="compliant"),
                          data =   {
                                      "DatasetProperties": {
                                          "FileSourceInfo": {
                                              "FileSource": storage_location['fileSource'],
                                              "Name": "image.jpeg"
                                          }
                                      },
                                      "Name": "Dataset with images",
                                      "ResourceSecurityClassification": f"{config['data-partition-id']}:reference-data--ResourceSecurityClassification:RESTRICTED:",
                                      "SchemaFormatTypeID": f"{config['data-partition-id']}:reference-data--SchemaFormatType:TabSeparatedColumnarText:"
                                  },
                          ancestry=RecordAncestry(parents=[]))
              ]

  registered_dataset = dataset_registry_client.register_dataset(CreateDatasetRegistriesRequest(dataset_registries=record_list)).json()

  pprint(storage_location)
  pprint(registered_dataset)

upload_file_and_metadata()

In [ ]:
# All to find there image and probably the other attendances images
query_request = QueryRequest(
    kind="*:*:*:*",
    query="data.Name:\"Dataset with images\"",
    limit=10,
    returned_fields=["*"],
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Show images in workbook and maybe on stage as they get uploaded
# ???

# Slot 2 - Investigating well data

#### All to search/list wells/wellbores/wellogs/trajectories/... that is uploaded in the **instances**

In [ ]:
# Wells
query_request = QueryRequest(
    kind="osdu:wks:master-data--Well:1.0.0", # "*:*:master-data--Well:*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Welllogs

query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellLog:*.*.*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Wellbores

query_request = QueryRequest(
    kind="osdu:wks:master-data--Wellbore:1.0.0",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

In [ ]:
# Trajectories

query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellboreTrajectory:*.*.*",
    query="",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())

#### For one of the few well/wellobres that we have prepped a bit

In [ ]:
# Use lineage to find welllogs/trajectories/... that belongs to a specific well/wellbore
# Welllogs

query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellLog:*.*.*",
    query="data.WellboreID:\"osdu:master-data--Wellbore:7233:\"",
    limit=5,
    returned_fields=["*"]
)
response = search_client.query_records(query_request)

pprint(response.json())


In [ ]:
# By usage show that the platform can handle and store versions for some of these data
record = Record(
    kind='osdu:wks:dataset--File.Generic:1.0.0',
    acl=Acl(owners=default_acl['owners'], viewers=default_acl['viewers']),
    legal=Legal(legaltags=default_legal_tags, other_relevant_data_countries=["US"], status="compliant"),
    data= {
        "msg": "Initial Message from AutoTest while testing update",
        "weight": 777.0
    })
# create
response = storage_client.create_update_records(records=[record])
stored_record = response.json()
recordId = stored_record['recordIds'][0]
pprint("the new record")
pprint(stored_record)

# get versions
versions = storage_client.get_record_versions(recordId=recordId).json()
pprint("current versions")
pprint(versions)

# update
record.data["weight"] = 778
record.id = recordId
response = storage_client.create_update_records(records=[])

# get versions
versions = storage_client.get_record_versions(recordId=recordId).json()
pprint("current versions")
pprint(versions)

# update
record.data["weight"] = 779
record.id = recordId
response = storage_client.create_update_records(records=[record])

# get versions
versions = storage_client.get_record_versions(recordId=recordId).json()
pprint("current versions")
pprint(versions)
checking_version = versions["versions"][2]

pprint(checking_version)

# get specific version
version = storage_client.get_specific_record(recordId=recordId, version=checking_version).json()
pprint("specific version")
pprint(version)

In [ ]:
# By usage show the difference between index/Storage/File APIs, for a data object
# In practice the user seeing the indexed data, the stored data and the payload data (CSV, LAS, DLIS, Parquet)

# upload file and metadata using dataset
upload_file_and_metadata()

# 10 sec for indexing
# search
query_request = QueryRequest(
    kind= "*:*:*:*",
    query="data.Name:\"Dataset with images\"",
    limit=2,
    returned_fields=["*"]
)
images_with_metadata = search_client.query_records(query_request).json()
pprint(images_with_metadata)

# get record from storage
record_id = images_with_metadata['results'][0]['id']
images_with_metadata = storage_client.query_record(recordId=record_id)
pprint(images_with_metadata.json())

# download using file service
file_host = config["FILE_HOST"]
url = f'https://{file_host}/files/{record_id}/downloadURL'
downloadUrlObject = requests.get(url, headers={'Data-Partition-Id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}).json()
pprint(downloadUrlObject)

file = requests.get(downloadUrlObject['SignedUrl'], headers={'Authorization': f'Bearer {token_refresher.access_token}'}, allow_redirects=True)
open('new_image.jpeg', 'wb').write(file.content)

# Slot 3 - Wellbore DDMS

For one of the few well/wellbores that we have prepped a bit

In [ ]:
# By usage see that the DDMS is an added capability to Index/Storage/File
# By usage see that same data can be extracted through both core and DDMS
# Show examples on using some of the DDMS APIs. To be determined
# Get basic well info to show that it's the same info as you get through the core APIs like (well header, wellbore header, ...)
# Demonstrate use of some added functionality in the DDMS
# Most likely related to logs and extraction of 1 specific curve, a specific curve interval, etc.

# Create a new well

url = f"https://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wells"
headers = {'data-partition-id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}
body = [{
        "acl": default_acl,
        "data": {
            "FacilityName": "Faciliity_AutoTest_999956686160",
            "ExtensionProperties": {
                "UWI": "20-000-00000-00"
            }
        },
        "id": f"{config['data-partition-id']}:master-data--Well:AutoTest_999956686160",
        "kind": "osdu:wks:master-data--Well:1.0.0",
        "legal": {
            "legaltags": default_legal_tags,
            "otherRelevantDataCountries": [
                "FR",
                "US"
            ],
            "status": "compliant"
        }
    }]


uploaded_well = requests.post(url, json=body, headers=headers).json()
pprint(uploaded_well)

# Get the well using Search
query_request = QueryRequest(
    kind="osdu:wks:master-data--Well:1.0.0",
    query="id:\"osdu:master-data--Well:AutoTest_999956686160\"",
    limit=5,
    returned_fields=["*"]
)
downloaded_well1 = search_client.query_records(query_request).json()
pprint(downloaded_well1)

# Get the well using Wellbore DDMS
url = f"https://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wells/{uploaded_well['recordIds'][0]}"
downloaded_well2 = requests.get(url, headers=headers).json()
pprint(downloaded_well2)


# Create a new wellbore

url = f"https://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellbores"
headers = {'data-partition-id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}
body = [{
        "acl": default_acl,
        "data": {
            "FacilityName": "Faciliity_AutoTest_999956686160",
            "ExtensionProperties": {
                "UWI": "20-000-00000-00"
            }
        },
        "id": f"{config['data-partition-id']}:master-data--Wellbore:AutoTest_999956686160",
        "kind": "osdu:wks:master-data--Wellbore:1.0.0",
        "legal": {
            "legaltags": default_legal_tags,
            "otherRelevantDataCountries": [
                "FR",
                "US"
            ]
        },
        "data": {
            "ExtensionProperties": {
                "airGap": {
                    "unitKey": "ft",
                    "value": 35.0
                },
                "country": "United States of America",
                "operator": "Francois Vinyes",
                "uwi": "33-089-00300-00-01",
                "wellboreType": "bypass"
            },
            "SpatialLocation": {
                "Wgs84Coordinates": {
                    "features": [{
                            "geometry": {
                                "coordinates": [
                                    [
                                        -103.2380248,
                                        46.8925081,
                                        5301.0
                                    ],
                                    [
                                        -103.2380248,
                                        46.8925081,
                                        2801.0
                                    ],
                                    [
                                        -103.2378748,
                                        46.892608100000004,
                                        301.0
                                    ],
                                    [
                                        -103.23742477750001,
                                        46.89270811,
                                        -2199.0
                                    ],
                                    [
                                        -103.23667470999663,
                                        46.892808120001,
                                        -4699.0
                                    ],
                                    [
                                        -103.2356245974865,
                                        46.892908130002,
                                        -7199.0
                                    ]
                                ],
                                "type": "LineString"
                            },
                            "properties": {
                                "name": "Newton 2-31-Lat-1"
                            },
                            "type": "Feature"
                        }
                    ],
                    "type": "FeatureCollection"
                }
            },
            "FacilityName": "Faciliity_AutoTest_99995668616",
            "WellID": "osdu:master-data--Well:AutoTest_999956686160:"
        },
        "meta": [{
                "kind": "Unit",
                "name": "Measure depth default unit",
                "persistableReference": "persistableReference",
                "propertyNames": ["symbol"]
            }
        ]
    }]

uploaded_wellbore = requests.post(url, json=body, headers=headers).json()
pprint(uploaded_wellbore)

# Get the wellbore using Search
query_request = QueryRequest(
    kind="osdu:wks:master-data--Wellbore:1.0.0",
    query="id:\"osdu:master-data--Wellbore:AutoTest_999956686160\"",
    limit=5,
    returned_fields=["*"]
)
downloaded_wellbore1 = search_client.query_records(query_request).json()
# pprint(downloaded_wellbore1)

# Get the wellbore using Wellbore DDMS
url = f"https://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellbores/{uploaded_wellbore['recordIds'][0]}"
downloaded_wellbore2 = requests.get(url, headers=headers).json()
pprint(downloaded_wellbore2)

# Create a trajectory
url = f"https://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellboretrajectories"
headers = {'data-partition-id': config['data-partition-id'], 'Authorization': f'Bearer {token_refresher.access_token}'}
body = [
    {
        "acl": default_acl,
        "legal": {
            "legaltags": default_legal_tags,
            "otherRelevantDataCountries": [
                "FR",
                "US"
            ]
        },
        "id": f"{config['data-partition-id']}:work-product-component--WellboreTrajectory:AutoTest_999956686160",
        "kind": f"{config['data-partition-id']}:wks:work-product-component--WellboreTrajectory:1.1.0",
        "data": {
            "Name": "Wellbore_Trajectory_AutoTest_999956686160",
            "WellboreID": f"{config['data-partition-id']}:master-data--Wellbore::AutoTest_999956686160:",
            "TopDepthMeasuredDepth": 0,
            "BaseDepthMeasuredDepth": 2.0,
            "VerticalMeasurement": {
                "EffectiveDateTime": "2021-08-17T14:13:08.174Z",
                "VerticalMeasurement": 0
            },
            "AvailableTrajectoryStationProperties": [
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:MD:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:m:",
                    "Name": "Measured Depth"
                },
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:INCL:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:dega:",
                    "Name": "Inclination"
                },
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:AZIM_TN:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:dega:",
                    "Name": "Azimuth"
                },
                {
                    "TrajectoryStationPropertyTypeID": f"{config['data-partition-id']}:reference-data--TrajectoryStationPropertyType:TVD:",
                    "StationPropertyUnitID": f"{config['data-partition-id']}:reference-data--UnitOfMeasure:m:",
                    "Name": "True Vertical Depth"
                }
            ]
        }
    }
]
uploaded_trajectories = requests.post(url, json=body, headers=headers).json()
pprint(uploaded_trajectories)

# Get the trajectory using Search
query_request = QueryRequest(
    kind="osdu:wks:work-product-component--WellboreTrajectory:1.1.0",
    query="id:\"osdu:work-product-component--WellboreTrajectory:AutoTest_999956686160\"",
    limit=5,
    returned_fields=["*"]
)
downloaded_trajectories1 = search_client.query_records(query_request).json()
pprint(downloaded_trajectories1)

# Get the trajectory using Wellbore DDMS
url = f"https://{config['WELLBORE_DDMS_HOST']}/ddms/v3/wellboretrajectories/{uploaded_trajectories['recordIds'][0]}"
downloaded_trajectories2 = requests.get(url, headers=headers).json()
pprint(downloaded_trajectories2)
